# GROMACS molekyylidynamiikkademonstraatio

Molekyylidynamiikka (MD) on monipuolinen laskennallinen menetelmä, jolla voidaan simuloida molekulaaristen systeemien kuten proteiinien ja lipidikalvojen liikettä. Tässä harjoituksessa demonstroidaan miten lipidien dynamiikkaa ja itsejärjestymistä voidaan tutkia [GROMACS](https://www.gromacs.org/)-molekyylidynamiikkaohjelmiston avulla. Simulaatioiden kiihdyttämiseksi hyödynnetään karkeistettua [Martini](http://cgmartini.nl/)-voimakenttään perustuvaa mallia, jossa useampaa vierekkäistä atomia mallinnetaan eräänlaisena "superatomina". Näin ollen simuloitavien hiukkasten määrä pienenee ja laskut nopeutuvat verrattuna tilanteeseen, jossa jokainen atomi kuvattaisiin eksplisiittisesti. Lisäksi simulaatiossa hyödynnetään rinnakkaislaskentaa suurteholaskennan (HPC) periaatteen demonstroimiseksi.

## Valmistelut

Ensiksi meidän tulee tehdä muutamia valintoja liittyen MD-simulaatioomme:

* Minkälaisia lipidejä haluamme tutkia? Entä kuinka montaa?
* Mitä liuotinta haluamme käyttää?
* Kuinka suurta (kuutiollista) simulaatiokoppia haluamme käyttää?

Aja konfiguraatioskripti alla valikon avaamiseksi, johon voit syöttää valintasi. Ehdotetut arvot ovat hyvä lähtökohta, jos teet harjoituksen ensimmäistä kertaa.

Tiedoksi, valittavia liuottimia (solvent) ovat W = vesi, EOL = etanoli ja OD = oktadekaani. Lipidien nimet ovat pitkiä ja monimutkaisia, joten lyhenteiden käyttö on yleistä. Simulaatiokopin koko (box size) on kuutiollisen kopin sivun pituus nanometreissä.

In [ ]:
%run config.py

Syöttämäsi parametrit tallentuvat muuttujiin `lipid.value`, `solvent.value`, `nlip.value` ja `box.value`.

## Solvatoidun lipidisysteemin rakentaminen

Solvatoitu lipidisysteemi rakennetaan seuraavaksi. Ensiksi lisätään valittu määrä valittua lipidiä kuutiolliseen simulaatiokoppiin, jonka sivun pituus määriteltiin yllä. Tämä tapahtuu GROMACS-komennon `insert-molecules` avulla ja saatu tulos tallennetaan muuttujaan `lipids.gro`.

In [ ]:
!gmx_mpi insert-molecules -o lipids.gro -ci ff22/{lipid.value}.gro -nmol {nlip.value} -box {box.value} -try 200

GROMACS tulostaa varsin paljon tietoa liittyen komennon suoritukseen. Oleellisin tieto on yleensä tulosteen lopussa. Tärkeintä on aina tarkistaa, ettei tulosteessa ole mainintaa virheistä (error). Jos saat virheen, lue virheviesti ja yritä pohtia mikä meni pieleen tai kysy apua.

Seuraavaksi meidän tulee lisätä valittu liuotin lipidilaatikkoomme. Alla oleva `solvate`-komento lisää koppiin niin paljon liuotinta kuin sinne mahtuu ottaen huomioon jo laatikossa olevat lipidimolekyylit. Saatu solvatoitu systeemi tallennetaan tiedostoon `solvated-lipids.gro`.

In [ ]:
!gmx_mpi solvate -cp lipids.gro -cs ff22/{solvent.value}.gro -o solvated-lipids.gro

**Hienoa, nyt sinulla on laatikko täynnä lipidejä ja valittua liuotinta!**

## Topologia

Ennen kuin voimme aloitta varsinaisen MD-simulaation, tulee meidän luoda nk. "topologia". Yksinkertaistaen, tämä on tiedosto, joka kertoo minkälaisia molekyylejä systeemissä on, kuinka monta, sekä mitä voimakenttää käytämmä näiden vuorovaikutuksen kuvaamiseksi.

Tässä harjoituksessa annetaan valmis mallitopologiatiedosto, mutta ennenkuin voimme käyttää sitä tulee meidän muokata tiedostoa molekyylien lukumäärän osalta. Eli etsi yllä olevista GROMACS-tulosteista kuinka monta lipidiä ja liuotinmolekyyliä mahtui oikeasti simulaatiokoppiin ja korvaa `LIPIDIEN_LUKUMÄÄRÄ` ja `LIUOTINMOLEKYYLIEN_LUKUMÄÄRÄ` alla olevassa komennossa näillä arvoilla.

Kun olet korvannut arvot, aja komento topologiatiedoston muokkaamiseksi.

**Huom! Jos lipidien ja liuotinmolekyylien lukumäärä topologiassa ei täsmää rakenteen kanssa, tulevat seuraavat askeleet epäonnistumaan, joten etsi ja korvaa arvot huolellisesti.**

In [ ]:
!bash init_files.sh {lipid.value} 50 {solvent.value} 1771

## Energian minimointi

Ennen MD-simulaatiota on hyvä varmistaa, että systeemin lähtögeometria on järkevä (atomit eivät ole liian lähellä toisiaan tai sidokset epäsuotuisasti vääntyneitä). Tämä tehdään minimoimalla systeemin energia. Käytännössä jokaiseen atomiin vaikuttava voima lasketaan voimakentän avulla ja atomeita liikutellaan voimien suuntaisesti kunnes ne saavuttavat minimin.

Luodaan syötetiedosto `em.tpr` energian minimointia varten, johon kootaan simulaatiparametrit (`em.mdp`), topologia (`topol.top`) ja systeemin rakenne (`solvated-lipids.gro`). Tämä tapahtuu automaattisesti alla olevan `grompp`-komennon avulla.

In [ ]:
!gmx_mpi grompp -f em.mdp -c solvated-lipids.gro -p topol.top -o em.tpr -maxwarn 10

Nyt meillä on input-tiedosto, joten aloitetaan energian minimointi `mdrun`-käskyllä.

In [ ]:
!gmx_mpi mdrun -s em.tpr -v

Voimme havainnollistaa potentiaalienergian pienenemisen. Aja komento alla, jotta saat tallennettua energian muuttumisen `ener.edr` output-tiedostosta tiedostoon nimeltä `energy.xvg`.

In [ ]:
!echo "5" | gmx_mpi energy -f ener.edr -o energy.xvg

Piirrä data `matplotlib`-työkalun avulla ajamalla alla oleva koodinpätkä.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

energy = np.loadtxt('energy.xvg', comments=['#', '@'])
plt.plot(energy[50:, 0], energy[50:, 1])
plt.xlabel('Energiaminimointiaskel')
plt.ylabel('Potentiaalienergia (kJ/mol)')
plt.show()

Kuvaajasta näemmä selkeästi miten systeemin potentiaalienergia pienenee ajon aikana. Nyt lähtögeometriamme on hieman stabiilimpi, joten olemme valmiit aloittamaan MD-simulaation syötetiedostojen valmistelun.

## Molekyylidynamiikka

Ennen MD-simulaatiota tulee meidän jällee valmistella syötetiedosto `md.tpr` kokoamalla yhteen energiaminimoitu rakenne (`confout.gro`), topologia (`topol.top`) ja MD-simulaation parametrit (`md.mdp`) `grompp`-käskyn avulla.

In [ ]:
!gmx_mpi grompp -f md.mdp -c confout.gro -p topol.top -o md.tpr -maxwarn 10

Molekyylidynamiikkasimulaatiota nopeutetaan ajamalla työ usen laskentaytimen avulla. Komento `orterun -n $SLURM_CPUS_PER_TASK` kertoo työtä käyttämään kaikkia laskentaytimiä, jotka varattiin kun tämä harjoitus käynnistettiin (4).

Riippuen systeemisi koosta ajo kestää muutamasta kymmenestä sekunnista muutamaan minuuttiin. Voit ajaa simulaation useamman kerran ja vaihtaa `$SLURM_CPUS_PER_TASK` johonkin pienempään arvoon. Miten tämä vaikuttaa simulaation nopeuteen? Tulosteen lopussa löytyy tiedot kuinka monta nanosekuntia per päivä (ns/day) kyseisisillä parametreilla voi simuloida.

Simulaation ajaminen hyödyntämällä useaa laskentaydintä on suurteholaskennan perusperiaate!

In [ ]:
!orterun -n $SLURM_CPUS_PER_TASK --oversubscribe gmx_mpi mdrun -deffnm md -v

## Visualization

Hienoa, olet suorittanut molekyylidynamiikkasimulaation valitsemillesi lipidimolekyyleille jossakin liuottimessa! Mielenkiintoisin kohta onkin nyt selvittää mitä simulaation aikana tapahtui.

Ennen tätä tulee meidän kuitenkin hieman korjata simulaation tuottamaa polkua `md.xtc` (aikakehitystä/trajectory). Ajamasi simulaatio hyödynsi nk. *periodisia reunaehtoja*, mikä tarkoittaa, että molekyylit voivat mennä simulaatiokopin reunojen läpi, jolloin ne päätyväy kopin vastakkaiselle puolelle. Tämä on erittäin kätevä tapa simuloida periodisia rakenteita kuten kiteitä ja liuoksia, mutta simulaation visualisointi voi antaa harhaanjohtavan kuvan, että molekyylit karkaavat toisistaan, johtaen pienenevään tiheyteen. Selkeän kuvan saamiseksi tulee meidän siis varmistaa, että reunat ylittävät molekyylit "käännetään" takaisin simulaatiokoppiin. Tämä tapahtuu seuraavan `trjconv`-käskyn avulla.

In [ ]:
!echo "0" | gmx_mpi trjconv -f md.xtc -s md.tpr -o md_wrap.xtc -pbc mol

Kokeile nyt visualisoida polku käyttäen `MDAnalysis`- ja `nglview`-työkaluja.

Visualisaatiossa harmaat pallot kuvaavat karkeistettuja atomeita, jotka kuuluvat lipidimolekyylien "häntiin", kun taas valkoiset ja värikkäät pallot kuvaavat pääryhmiin kuuluvia atomeita. Pienet pisteet ovat liuotinmolekyylejä.

In [ ]:
import MDAnalysis as mda
import nglview as nv

u = mda.Universe('md.gro', 'md_wrap.xtc')
view = nv.show_mdanalysis(u)
view.add_spacefill(lipid.value)
view.add_point(solvent.value)
view.add_unitcell()
view

Kuten näet, pysyvät atomit simulaatiokopin sisällä koko simulaation aikana. Tekemämme korjaus saattaa kuitenkin aiheuttaa sen, että lipidien muodostama rakenne katkeaa reunojen yli, mikä hankaloittaa rakenteen hahmottamista. Kokeile ajaa alla oleva komento, joka siirtää lipidirakenteen kopin keskelle. Visualisoi polku sitten uudelleen (tällä kertaa ilman liuotinmolekyylejä).

In [ ]:
!echo "2 2 0" | gmx_mpi trjconv -f md.xtc -s md.tpr -o md_whole.xtc -pbc cluster -center

In [ ]:
import MDAnalysis as mda
import nglview as nv

u = mda.Universe('md.gro', 'md_whole.xtc')
view = nv.show_mdanalysis(u)
view.add_spacefill(lipid.value)
view.add_unitcell()
view

## Termodynaaminen ja geometrinen tulkinta

Eli mitä tässä oikein tapahtui? Simulaation alussa lipidit ovat siellä täällä, mutta ajan myötä ne vaikuttavat järjestyvän. Ilmiölle on termodynaaminen tulkinta. Lipidit ovat ns. amfifiilisiä molekyylejä, eli niillä on hydrofiilininen (poolinen) pääryhmä sekä hydrofobinen (pooliton) häntä. Riippuen liuottiminen poolisuudesta lipidit haluavat siis järjestyä siten, että saman poolisuuden omaavat ryhmät ovat lähellä toisiaan. 

Lipidien itsejärjestyminen minimoi systeemin *vapaaenergian* $\Delta G$. Koska prosessi on spontaani (se tapahtuu itsestään), täytyy systeemin vapaaenergian muutoksen olla negatiivinen. Yleensä termodynaamiset systeemit pyrkivät kasvattamaan entropiaan ("epäjärjestystä"), mutta tässä tapauksessa systeemi järjestyy, eli entropia pienenee! Vapaaenergiaan vaikuttaa kuitenkin olennaisesti myös entalpia, jonka muuton erittäin negatiivinen sillä lipidien järjestyminen poolisuuden perusteella johtaa optimaalisempaan elektrostaattiseen vuorovaikutukseen ryhmien välillä. Näin ollen vapaaenergian muutos on kokonaisuudessaan negatiivinen,

$$\Delta G=\Delta H-T\Delta S<0$$

Liuottimen määräämän järestyksen lisäksi lipidien geometria vaikuttaa siihen minkälaisia rakenteita muodostuu. Kriittinen pakkautumisparametri (CPP) on tyypillisesti käytetty mitta, jolla voidaan ennustaa laadullisesti itsejärjestyneen lipidiagregaatin muoto,

$$\mathrm{CPP}=\frac{V_t}{A_h\cdot l_t}$$

missä $V_t$ ja $l_t$ ovat hännän tilavuus ja pituus, kun taas $A_h$ pääryhmän poikkileikkauksen pinta-ala. Olettaen poolinen liuotin (vesi tai etanoli), lipidit joilla on iso pääryhmä ($\mathrm{CPP}\leq1/3$) muodostavat tyypillisesti pallomaisia *misellejä*, kun taas lieriömäiset rakenteet ja *vesikkelit*, eli pallomaiset kaksoiskalvot, ovat mahdollisia jos pääryhmä on hieman pienempi ($1/3<\mathrm{CPP}\leq1/2$, esim. LPC). Jos lipidi on muodoltaan lieriö (pään poikkileikkaus on n. yhtä suuri kuin hännän, $1/2<\mathrm{CPP}\leq1$, esim. DPPC), voi erinäisiä kaksoiskalvoja muodostua.

Toisaalta, poolittomissa liuottimissa lipidi voivat myös muodostaa invertoituneita misellejä, missä hännät osoittavat kohti liuotinta. Inversio on mahdollista myös poolisissa liuottimissa jos lipidien pääryhmä on hyvin pieni ($\mathrm{CPP}>1$, esim. POPE). Tällöin itsejärjestyneissä rakenteissa liuotin on käytännössä vangittu hydrofiilisten pääryhmien muodostamiin aukkoihin.

## Kokeile simulaatiota uudestaan

Kokeile muuttaa simuloitua lipidiä ja/tai luotinta ja tutki saatko tuotettua joitain yllä mainituista itsejärjestyneistä rakenteista. Huomaa, että myös lipidien konsentraatio vaikuttaa tuloksiin -- isojen rakenteiden kuten kaksoiskalvojen muodostuminen on epätodennäköistä jos lipidimolekyylien lukumäärä on liian pieni suhteessta simulaatiokopin kokoon.

Aloittaaksesi puhtaalta pöydältä voit ajaa alla olevan skriptin poistaaksesi kaikki tähän asti luodut tiedostot.

In [ ]:
!bash clean.sh